In [1]:
#Base

import pandas as pd
import os

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SMOTENC
from imblearn.over_sampling import ADASYN
from imblearn.combine import SMOTETomek

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE

import statsmodels.api as sm
from tqdm import tqdm

dataset_folder = os.path.join(os.getcwd(), 'dataset')

def df_from_csv(filename):
    f = os.path.join(dataset_folder, filename)
    return pd.read_csv(f, delimiter='\t|\n|,', engine='python')

x_train = df_from_csv('ori_trainx.csv')
y_train = df_from_csv('ori_trainy.csv')
alert_date = df_from_csv('ori_alert_date.csv')
alert_cust = df_from_csv('ori_custinfo.csv')

smote = SMOTE(random_state=0)
tqdm.pandas(desc='Progress Bar')

def RFE_analysis(input_x, input_y, model):
    rfe = RFE(model, step=300, n_features_to_select=0.97)
    rfe.fit(input_x, input_y.values.ravel())
    print(input_x.columns)
    print(rfe.support_)
    print(rfe.ranking_)
    return rfe.get_support(1)

def LR_analysis(input_x, input_y):
    sm_LR = sm.Logit(input_y, input_x)
    result = sm_LR.fit()
    print(result.summary2())

def LR_training(input_x, input_y, LR_model):
    x_train, x_test, y_train, y_test = train_test_split(input_x, input_y, test_size=0.2, random_state=42)
    LR_model.fit(x_train, y_train)
    y_train_predict = LR_model.predict(x_train)
    y_test_predict = LR_model.predict(x_test)
    train_acc = accuracy_score(y_train, y_train_predict)
    test_acc = accuracy_score(y_test, y_test_predict)

    print(f'Train acc: {train_acc}  Test acc:{test_acc}')
    cm_train = confusion_matrix(y_true=y_train, y_pred=y_train_predict)
    print(cm_train)
    print(classification_report(y_true=y_train, y_pred=y_train_predict))

    print(f'\nTest acc: {train_acc}  Test acc:{test_acc}')
    cm_test = confusion_matrix(y_true=y_test, y_pred=y_test_predict)
    print(cm_test)
    print(classification_report(y_true=y_test, y_pred=y_test_predict))

    return LR_model

def LR_test(input_x, input_y, LR_model):
    y_predict = LR_model.predict(input_x)
    test_acc = accuracy_score(input_y, y_predict)
    print(f'Test acc: {test_acc}')
    cm = confusion_matrix(y_true=input_y, y_pred=y_predict)
    print(cm)
    print(classification_report(y_true=input_y, y_pred=y_predict))

def calculate_tx(row):
    if not pd.isna(row['risk_rank']):
        temp_frame = xdp[(xdp['tx_date'] <= row['date']) & (xdp['cust_id'] == row['cust_id'])]
        total_tx = (temp_frame['tx_amt'] * temp_frame['exchg_rate'])
        row['total_tx'] = total_tx.sum()
        row['avg_tx'] = total_tx.mean()
        row['count_tx'] = total_tx.count()
        row['std_tx'] = total_tx.std()
        row['tx_per_day'] = row['count_tx']/row['date']+1
    return row

In [2]:
# Join data for training

ccba = pd.read_pickle('aggregated_ccba_cust.pkl')
cdtx = pd.read_pickle('alert_key_cdtx.pkl')
xdp = pd.read_pickle('xdp_x_train.pkl')
remit = pd.read_pickle('cust_remit_raw_pre_20221221.pkl')

In [3]:
# Delete bad columns in one of dataset

remit = remit.iloc[:, [0, 1, 2, 3, 4, 5, 7]]
remit

,alert_key,total_asset_x,sum_assets,total_asset_y,trade_sum,count_assets,ratio
0,352249,1465816.0,33853238.0,1.471880e+06,NaN,23,NaN
1,352253,98177.0,4997197.0,8.923566e+04,2923745.0,56,0.030521
2,352254,2052922.0,2052922.0,2.052922e+06,NaN,1,NaN
3,352280,201906.0,3529876.0,6.660143e+04,164364.0,53,0.405207
4,352282,7450.0,14900.0,7.450000e+03,8356.0,2,0.891575
...,...,...,...,...,...,...,...
25746,352123,12207.0,427479.0,1.042632e+04,NaN,41,NaN
25747,352124,259985.0,3490135.0,1.517450e+05,NaN,23,NaN
25748,352125,928963.0,1008265.0,5.041325e+05,22127.0,2,22.783590
25749,352128,21647.0,902254.0,3.007513e+04,NaN,30,NaN


In [4]:
train = df_from_csv('ori_trainx.csv')
train = train.merge(df_from_csv('ori_trainy.csv'), on='alert_key', how='inner')
train = train.merge(alert_cust, on='alert_key', how='left')
train = train[['alert_key', 'cust_id']]
train = train.merge(ccba, on='cust_id', how='left')
train = train.merge(cdtx, on='alert_key', how='left')
train = train.merge(xdp, on='alert_key', how='left')
train = train.merge(remit, on='alert_key', how='left')
train = train.reindex(sorted(train.columns), axis=1)
train

,AGE,alert_key,atm_count_rate,atm_tx_rate,avg_amt_nontw_nonntd,avg_amt_nontw_ntd,avg_amt_tw_nonntd,avg_amt_tw_ntd,avg_freq_nontw_nonntd,avg_freq_nontw_ntd,...,total_amt_tw_nonntd,total_amt_tw_ntd,total_asset,total_asset_x,total_asset_y,total_freq_nontw_nonntd,total_freq_nontw_ntd,total_freq_tw_nonntd,total_freq_tw_ntd,trade_sum
0,4,171189,1.000000,1.000000,NaN,673.00000,NaN,NaN,NaN,1.000000,...,0.0,0.0,375576.0,375576.0,4.886652e+05,0,1,0,0,NaN
1,2,171202,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,2717416.0,2717416.0,2.621840e+06,0,0,0,0,NaN
2,4,171599,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,326517.0,326517.0,3.265170e+05,0,0,0,0,NaN
3,4,171737,0.900000,0.969090,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,1014759.0,1014759.0,1.275499e+05,0,0,0,0,NaN
4,3,171142,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,241719.0,241719.0,2.212473e+05,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23901,3,352132,0.690141,0.977016,4400.571429,1242.34238,NaN,1013.510067,1.166667,2.371287,...,0.0,2416208.0,3218731.0,3218731.0,2.361632e+06,63,479,0,2384,364459.0
23902,3,352125,0.650000,0.608263,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,928963.0,928963.0,5.041325e+05,0,0,0,0,22127.0
23903,6,352080,0.600000,0.941020,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,69080.0,69080.0,7.500171e+04,0,0,0,0,43676.0
23904,6,352075,0.969767,0.998672,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,262604.0,262604.0,3.333440e+05,0,0,0,0,NaN


In [5]:
train['bad_value'] = train.isnull().sum(axis=1) + (train == 0).astype(int).sum(axis=1)

In [6]:
train = train.drop(['cust_id_x', 'cust_id_y', 'date'], axis=1)
train.to_pickle('aggregate.pkl')

In [7]:
train = train.drop(['alert_key'], axis=1)
train

,AGE,atm_count_rate,atm_tx_rate,avg_amt_nontw_nonntd,avg_amt_nontw_ntd,avg_amt_tw_nonntd,avg_amt_tw_ntd,avg_freq_nontw_nonntd,avg_freq_nontw_ntd,avg_freq_tw_nonntd,...,total_amt_tw_ntd,total_asset,total_asset_x,total_asset_y,total_freq_nontw_nonntd,total_freq_nontw_ntd,total_freq_tw_nonntd,total_freq_tw_ntd,trade_sum,bad_value
0,4,1.000000,1.000000,NaN,673.00000,NaN,NaN,NaN,1.000000,NaN,...,0.0,375576.0,375576.0,4.886652e+05,0,1,0,0,NaN,86
1,2,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,2717416.0,2717416.0,2.621840e+06,0,0,0,0,NaN,85
2,4,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,326517.0,326517.0,3.265170e+05,0,0,0,0,NaN,90
3,4,0.900000,0.969090,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1014759.0,1014759.0,1.275499e+05,0,0,0,0,NaN,86
4,3,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,241719.0,241719.0,2.212473e+05,0,0,0,0,NaN,83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23901,3,0.690141,0.977016,4400.571429,1242.34238,NaN,1013.510067,1.166667,2.371287,NaN,...,2416208.0,3218731.0,3218731.0,2.361632e+06,63,479,0,2384,364459.0,58
23902,3,0.650000,0.608263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,928963.0,928963.0,5.041325e+05,0,0,0,0,22127.0,78
23903,6,0.600000,0.941020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,69080.0,69080.0,7.500171e+04,0,0,0,0,43676.0,77
23904,6,0.969767,0.998672,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,262604.0,262604.0,3.333440e+05,0,0,0,0,NaN,72


In [8]:
train['bad_value'] = train.isnull().sum(axis=1) + (train == 0).astype(int).sum(axis=1)
train

,AGE,atm_count_rate,atm_tx_rate,avg_amt_nontw_nonntd,avg_amt_nontw_ntd,avg_amt_tw_nonntd,avg_amt_tw_ntd,avg_freq_nontw_nonntd,avg_freq_nontw_ntd,avg_freq_tw_nonntd,...,total_amt_tw_ntd,total_asset,total_asset_x,total_asset_y,total_freq_nontw_nonntd,total_freq_nontw_ntd,total_freq_tw_nonntd,total_freq_tw_ntd,trade_sum,bad_value
0,4,1.000000,1.000000,NaN,673.00000,NaN,NaN,NaN,1.000000,NaN,...,0.0,375576.0,375576.0,4.886652e+05,0,1,0,0,NaN,85
1,2,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,2717416.0,2717416.0,2.621840e+06,0,0,0,0,NaN,84
2,4,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,326517.0,326517.0,3.265170e+05,0,0,0,0,NaN,89
3,4,0.900000,0.969090,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1014759.0,1014759.0,1.275499e+05,0,0,0,0,NaN,85
4,3,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,241719.0,241719.0,2.212473e+05,0,0,0,0,NaN,82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23901,3,0.690141,0.977016,4400.571429,1242.34238,NaN,1013.510067,1.166667,2.371287,NaN,...,2416208.0,3218731.0,3218731.0,2.361632e+06,63,479,0,2384,364459.0,58
23902,3,0.650000,0.608263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,928963.0,928963.0,5.041325e+05,0,0,0,0,22127.0,78
23903,6,0.600000,0.941020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,69080.0,69080.0,7.500171e+04,0,0,0,0,43676.0,77
23904,6,0.969767,0.998672,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,262604.0,262604.0,3.333440e+05,0,0,0,0,NaN,72


In [9]:
ori_train = train.copy()
train

,AGE,atm_count_rate,atm_tx_rate,avg_amt_nontw_nonntd,avg_amt_nontw_ntd,avg_amt_tw_nonntd,avg_amt_tw_ntd,avg_freq_nontw_nonntd,avg_freq_nontw_ntd,avg_freq_tw_nonntd,...,total_amt_tw_ntd,total_asset,total_asset_x,total_asset_y,total_freq_nontw_nonntd,total_freq_nontw_ntd,total_freq_tw_nonntd,total_freq_tw_ntd,trade_sum,bad_value
0,4,1.000000,1.000000,NaN,673.00000,NaN,NaN,NaN,1.000000,NaN,...,0.0,375576.0,375576.0,4.886652e+05,0,1,0,0,NaN,85
1,2,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,2717416.0,2717416.0,2.621840e+06,0,0,0,0,NaN,84
2,4,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,326517.0,326517.0,3.265170e+05,0,0,0,0,NaN,89
3,4,0.900000,0.969090,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1014759.0,1014759.0,1.275499e+05,0,0,0,0,NaN,85
4,3,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,241719.0,241719.0,2.212473e+05,0,0,0,0,NaN,82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23901,3,0.690141,0.977016,4400.571429,1242.34238,NaN,1013.510067,1.166667,2.371287,NaN,...,2416208.0,3218731.0,3218731.0,2.361632e+06,63,479,0,2384,364459.0,58
23902,3,0.650000,0.608263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,928963.0,928963.0,5.041325e+05,0,0,0,0,22127.0,78
23903,6,0.600000,0.941020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,69080.0,69080.0,7.500171e+04,0,0,0,0,43676.0,77
23904,6,0.969767,0.998672,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,262604.0,262604.0,3.333440e+05,0,0,0,0,NaN,72


In [10]:
ori_train.iloc[:, 1:] = ori_train.iloc[:, 1:].fillna(0)
train.iloc[:, 1:] = train.iloc[:, 1:].fillna(0)
train

,AGE,atm_count_rate,atm_tx_rate,avg_amt_nontw_nonntd,avg_amt_nontw_ntd,avg_amt_tw_nonntd,avg_amt_tw_ntd,avg_freq_nontw_nonntd,avg_freq_nontw_ntd,avg_freq_tw_nonntd,...,total_amt_tw_ntd,total_asset,total_asset_x,total_asset_y,total_freq_nontw_nonntd,total_freq_nontw_ntd,total_freq_tw_nonntd,total_freq_tw_ntd,trade_sum,bad_value
0,4,1.000000,1.000000,0.000000,673.00000,0.0,0.000000,0.000000,1.000000,0.0,...,0.0,375576.0,375576.0,4.886652e+05,0,1,0,0,0.0,85
1,2,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,2717416.0,2717416.0,2.621840e+06,0,0,0,0,0.0,84
2,4,1.000000,1.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,326517.0,326517.0,3.265170e+05,0,0,0,0,0.0,89
3,4,0.900000,0.969090,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,1014759.0,1014759.0,1.275499e+05,0,0,0,0,0.0,85
4,3,1.000000,1.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,241719.0,241719.0,2.212473e+05,0,0,0,0,0.0,82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23901,3,0.690141,0.977016,4400.571429,1242.34238,0.0,1013.510067,1.166667,2.371287,0.0,...,2416208.0,3218731.0,3218731.0,2.361632e+06,63,479,0,2384,364459.0,58
23902,3,0.650000,0.608263,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,928963.0,928963.0,5.041325e+05,0,0,0,0,22127.0,78
23903,6,0.600000,0.941020,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,69080.0,69080.0,7.500171e+04,0,0,0,0,43676.0,77
23904,6,0.969767,0.998672,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,262604.0,262604.0,3.333440e+05,0,0,0,0,0.0,72


In [11]:
ori_train = ori_train.drop(['bad_value'], axis=1)
train = train.drop(['bad_value'], axis=1)
train

,AGE,atm_count_rate,atm_tx_rate,avg_amt_nontw_nonntd,avg_amt_nontw_ntd,avg_amt_tw_nonntd,avg_amt_tw_ntd,avg_freq_nontw_nonntd,avg_freq_nontw_ntd,avg_freq_tw_nonntd,...,total_amt_tw_nonntd,total_amt_tw_ntd,total_asset,total_asset_x,total_asset_y,total_freq_nontw_nonntd,total_freq_nontw_ntd,total_freq_tw_nonntd,total_freq_tw_ntd,trade_sum
0,4,1.000000,1.000000,0.000000,673.00000,0.0,0.000000,0.000000,1.000000,0.0,...,0.0,0.0,375576.0,375576.0,4.886652e+05,0,1,0,0,0.0
1,2,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,2717416.0,2717416.0,2.621840e+06,0,0,0,0,0.0
2,4,1.000000,1.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,326517.0,326517.0,3.265170e+05,0,0,0,0,0.0
3,4,0.900000,0.969090,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,1014759.0,1014759.0,1.275499e+05,0,0,0,0,0.0
4,3,1.000000,1.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,241719.0,241719.0,2.212473e+05,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23901,3,0.690141,0.977016,4400.571429,1242.34238,0.0,1013.510067,1.166667,2.371287,0.0,...,0.0,2416208.0,3218731.0,3218731.0,2.361632e+06,63,479,0,2384,364459.0
23902,3,0.650000,0.608263,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,928963.0,928963.0,5.041325e+05,0,0,0,0,22127.0
23903,6,0.600000,0.941020,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,69080.0,69080.0,7.500171e+04,0,0,0,0,43676.0
23904,6,0.969767,0.998672,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,262604.0,262604.0,3.333440e+05,0,0,0,0,0.0


In [12]:
# Training Dataset
ori_train_x = ori_train.loc[:, train.columns != 'sar_flag']
ori_train_y = ori_train.loc[:, train.columns == 'sar_flag']
train_x = train.loc[:, train.columns != 'sar_flag']
train_y = train.loc[:, train.columns == 'sar_flag']
train_x

,AGE,atm_count_rate,atm_tx_rate,avg_amt_nontw_nonntd,avg_amt_nontw_ntd,avg_amt_tw_nonntd,avg_amt_tw_ntd,avg_freq_nontw_nonntd,avg_freq_nontw_ntd,avg_freq_tw_nonntd,...,total_amt_tw_nonntd,total_amt_tw_ntd,total_asset,total_asset_x,total_asset_y,total_freq_nontw_nonntd,total_freq_nontw_ntd,total_freq_tw_nonntd,total_freq_tw_ntd,trade_sum
0,4,1.000000,1.000000,0.000000,673.00000,0.0,0.000000,0.000000,1.000000,0.0,...,0.0,0.0,375576.0,375576.0,4.886652e+05,0,1,0,0,0.0
1,2,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,2717416.0,2717416.0,2.621840e+06,0,0,0,0,0.0
2,4,1.000000,1.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,326517.0,326517.0,3.265170e+05,0,0,0,0,0.0
3,4,0.900000,0.969090,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,1014759.0,1014759.0,1.275499e+05,0,0,0,0,0.0
4,3,1.000000,1.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,241719.0,241719.0,2.212473e+05,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23901,3,0.690141,0.977016,4400.571429,1242.34238,0.0,1013.510067,1.166667,2.371287,0.0,...,0.0,2416208.0,3218731.0,3218731.0,2.361632e+06,63,479,0,2384,364459.0
23902,3,0.650000,0.608263,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,928963.0,928963.0,5.041325e+05,0,0,0,0,22127.0
23903,6,0.600000,0.941020,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,69080.0,69080.0,7.500171e+04,0,0,0,0,43676.0
23904,6,0.969767,0.998672,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,262604.0,262604.0,3.333440e+05,0,0,0,0,0.0


In [13]:
# Normalize function
from sklearn import preprocessing

def normalize_mms(input_df):
    scaler = preprocessing.MinMaxScaler()
    result = scaler.fit_transform(input_df)
    return pd.DataFrame(result, index=input_df.index, columns=input_df.columns)

In [14]:
train_x = normalize_mms(train_x)
train_x

,AGE,atm_count_rate,atm_tx_rate,avg_amt_nontw_nonntd,avg_amt_nontw_ntd,avg_amt_tw_nonntd,avg_amt_tw_ntd,avg_freq_nontw_nonntd,avg_freq_nontw_ntd,avg_freq_tw_nonntd,...,total_amt_tw_nonntd,total_amt_tw_ntd,total_asset,total_asset_x,total_asset_y,total_freq_nontw_nonntd,total_freq_nontw_ntd,total_freq_tw_nonntd,total_freq_tw_ntd,trade_sum
0,0.4,1.000000,1.000000,0.000000,0.004125,0.0,0.000000,0.00000,0.038462,0.0,...,0.0,0.000000,0.005085,0.005085,0.009321,0.000000,0.000402,0.0,0.00000,0.000000
1,0.2,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.0,...,0.0,0.000000,0.036790,0.036790,0.050010,0.000000,0.000000,0.0,0.00000,0.000000
2,0.4,1.000000,1.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.0,...,0.0,0.000000,0.004421,0.004421,0.006228,0.000000,0.000000,0.0,0.00000,0.000000
3,0.4,0.900000,0.969090,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.0,...,0.0,0.000000,0.013738,0.013738,0.002433,0.000000,0.000000,0.0,0.00000,0.000000
4,0.3,1.000000,1.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.0,...,0.0,0.000000,0.003273,0.003273,0.004220,0.000000,0.000000,0.0,0.00000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23901,0.3,0.690141,0.977016,0.001785,0.007614,0.0,0.000024,0.00334,0.091203,0.0,...,0.0,0.018918,0.043577,0.043577,0.045047,0.002654,0.192679,0.0,0.20534,0.008875
23902,0.3,0.650000,0.608263,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.0,...,0.0,0.000000,0.012577,0.012577,0.009616,0.000000,0.000000,0.0,0.00000,0.000539
23903,0.6,0.600000,0.941020,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.0,...,0.0,0.000000,0.000935,0.000935,0.001431,0.000000,0.000000,0.0,0.00000,0.001064
23904,0.6,0.969767,0.998672,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.0,...,0.0,0.000000,0.003555,0.003555,0.006358,0.000000,0.000000,0.0,0.00000,0.000000


In [15]:
# View Categorial Feature
attribute_list = list(train_x.columns)
for i, v in enumerate(attribute_list):
    print(f'{i} {v}')

0 AGE
1 atm_count_rate
2 atm_tx_rate
3 avg_amt_nontw_nonntd
4 avg_amt_nontw_ntd
5 avg_amt_tw_nonntd
6 avg_amt_tw_ntd
7 avg_freq_nontw_nonntd
8 avg_freq_nontw_ntd
9 avg_freq_tw_nonntd
10 avg_freq_tw_ntd
11 cb_count_rate
12 cb_tx_rate
13 count_assets
14 cr_1_avg_tx
15 cr_1_count_tx
16 cr_1_std_tx
17 cr_1_total_tx
18 cr_1_tx_per_day
19 cr_2_avg_tx
20 cr_2_count_tx
21 cr_2_std_tx
22 cr_2_total_tx
23 cr_2_tx_per_day
24 cr_3_avg_tx
25 cr_3_count_tx
26 cr_3_std_tx
27 cr_3_total_tx
28 cr_3_tx_per_day
29 cucah_quarter1_mean
30 cucah_quarter1_std
31 cucah_quarter1_sum
32 cucah_quarter2_mean
33 cucah_quarter2_std
34 cucah_quarter2_sum
35 cucah_quarter3_mean
36 cucah_quarter3_std
37 cucah_quarter3_sum
38 cucah_quarter4_mean
39 cucah_quarter4_std
40 cucah_quarter4_sum
41 cucah_total_mean
42 cucah_total_std
43 cucah_total_sum
44 db_1_avg_tx
45 db_1_count_tx
46 db_1_std_tx
47 db_1_total_tx
48 db_1_tx_per_day
49 db_2_avg_tx
50 db_2_count_tx
51 db_2_std_tx
52 db_2_total_tx
53 db_2_tx_per_day
54 db_3_av

In [16]:
cf_index = [0] + list(range(59,80))
cf_index

[0,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79]

In [17]:
smote = SMOTE(random_state=42)
smn = SMOTENC(categorical_features=cf_index)
smt = SMOTETomek(smote=smn, random_state=42)
ada = ADASYN(random_state=42)
sm_x, sm_y = ada.fit_resample(train_x, train_y)
sm1_x, sm1_y = smote.fit_resample(train_x, train_y)
sm2_x, sm2_y = smt.fit_resample(train_x, train_y)

#sm = adasyn, sm1 = smote, sm2 = smotetomek

In [18]:
LR = LogisticRegression(random_state=42)
RF = RandomForestClassifier(random_state=42)

In [19]:
sm_x

,AGE,atm_count_rate,atm_tx_rate,avg_amt_nontw_nonntd,avg_amt_nontw_ntd,avg_amt_tw_nonntd,avg_amt_tw_ntd,avg_freq_nontw_nonntd,avg_freq_nontw_ntd,avg_freq_tw_nonntd,...,total_amt_tw_nonntd,total_amt_tw_ntd,total_asset,total_asset_x,total_asset_y,total_freq_nontw_nonntd,total_freq_nontw_ntd,total_freq_tw_nonntd,total_freq_tw_ntd,trade_sum
0,0.40000,1.000000,1.000000,0.0,4.124798e-03,0.0,0.000000e+00,0.0,0.038462,0.0,...,0.0,0.000000e+00,0.005085,0.005085,0.009321,0.0,4.022526e-04,0.0,0.000000e+00,0.000000e+00
1,0.20000,0.000000,0.000000,0.0,0.000000e+00,0.0,0.000000e+00,0.0,0.000000,0.0,...,0.0,0.000000e+00,0.036790,0.036790,0.050010,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00
2,0.40000,1.000000,1.000000,0.0,0.000000e+00,0.0,0.000000e+00,0.0,0.000000,0.0,...,0.0,0.000000e+00,0.004421,0.004421,0.006228,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00
3,0.40000,0.900000,0.969090,0.0,0.000000e+00,0.0,0.000000e+00,0.0,0.000000,0.0,...,0.0,0.000000e+00,0.013738,0.013738,0.002433,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00
4,0.30000,1.000000,1.000000,0.0,0.000000e+00,0.0,0.000000e+00,0.0,0.000000,0.0,...,0.0,0.000000e+00,0.003273,0.003273,0.004220,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47292,0.20000,0.212815,0.361712,0.0,1.753813e-07,0.0,5.973528e-08,0.0,0.000012,0.0,...,0.0,5.177058e-07,0.001917,0.001917,0.002701,0.0,1.264891e-07,0.0,7.041990e-07,3.750706e-08
47293,0.20000,0.323894,0.400762,0.0,0.000000e+00,0.0,0.000000e+00,0.0,0.000000,0.0,...,0.0,0.000000e+00,0.000595,0.000595,0.000839,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00
47294,0.20000,0.469377,0.637090,0.0,3.505811e-04,0.0,1.194087e-04,0.0,0.024176,0.0,...,0.0,1.034876e-03,0.013702,0.013702,0.019305,0.0,2.528473e-04,0.0,1.407669e-03,7.497529e-05
47295,0.20000,0.417370,0.581269,0.0,2.795516e-04,0.0,9.521594e-05,0.0,0.019278,0.0,...,0.0,8.252048e-04,0.011313,0.011313,0.015939,0.0,2.016192e-04,0.0,1.122468e-03,5.978493e-05


### 1 Model Comparison, without Oversampling

##### 1.1 Logistic Regression - Train & Test Accuracy

Note: High Accuracy but worst recall & precision (0 Recall & Precision)
- Bad training precision recall
- Bad test precision recall

In [20]:
LR = LR_training(train_x, train_y.values.ravel(), LR)

Train acc: 0.9907969044133027  Test acc:0.9878711836051861
[[18948     0]
 [  176     0]]
              precision    recall  f1-score   support

           0       0.99      1.00      1.00     18948
           1       0.00      0.00      0.00       176

    accuracy                           0.99     19124
   macro avg       0.50      0.50      0.50     19124
weighted avg       0.98      0.99      0.99     19124


Test acc: 0.9907969044133027  Test acc:0.9878711836051861
[[4724    0]
 [  58    0]]
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      4724
           1       0.00      0.00      0.00        58

    accuracy                           0.99      4782
   macro avg       0.49      0.50      0.50      4782
weighted avg       0.98      0.99      0.98      4782



C:\Software\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Software\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Software\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Software\Anaconda3\lib\site-packages\sklearn\metrics\_classif

##### 1.2 Random Forest - Test Accuracy

Note: High Accuracy but Not bad precision, but very bad recall (At least not 0 but very bad)

- Best training precision recall
- Bad test precision recall

In [21]:
RF = LR_training(train_x, train_y, RF)

C:\Users\Edwin Sanjaya\AppData\Local\Temp\ipykernel_1296\528896171.py:52: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  LR_model.fit(x_train, y_train)


Train acc: 1.0  Test acc:0.9882894186532831
[[18948     0]
 [    0   176]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18948
           1       1.00      1.00      1.00       176

    accuracy                           1.00     19124
   macro avg       1.00      1.00      1.00     19124
weighted avg       1.00      1.00      1.00     19124


Test acc: 1.0  Test acc:0.9882894186532831
[[4722    2]
 [  54    4]]
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      4724
           1       0.67      0.07      0.12        58

    accuracy                           0.99      4782
   macro avg       0.83      0.53      0.56      4782
weighted avg       0.98      0.99      0.98      4782



#### 2 Model Comparison, with SMOTE oversampling

##### 2.1a Logistic Regression with SMOTE - Train & Test Accuracy (Oversampled Data)

Better training & test in oversampled data

In [22]:
LR = LR_training(sm1_x, sm1_y.values.ravel(), LR)

Train acc: 0.8845676567656766  Test acc:0.8898510930404477
[[16037  2916]
 [ 1456 17466]]
              precision    recall  f1-score   support

           0       0.92      0.85      0.88     18953
           1       0.86      0.92      0.89     18922

    accuracy                           0.88     37875
   macro avg       0.89      0.88      0.88     37875
weighted avg       0.89      0.88      0.88     37875


Test acc: 0.8845676567656766  Test acc:0.8898510930404477
[[4039  680]
 [ 363 4387]]
              precision    recall  f1-score   support

           0       0.92      0.86      0.89      4719
           1       0.87      0.92      0.89      4750

    accuracy                           0.89      9469
   macro avg       0.89      0.89      0.89      9469
weighted avg       0.89      0.89      0.89      9469



C:\Software\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##### 2.1b Logistic Regression with SMOTE - Accuracy (Real non-oversampled data)

Good recall but bad precision

In [23]:
LR_test(train_x, train_y.values.ravel(), LR)

Test acc: 0.8483644273404166
[[20076  3596]
 [   29   205]]
              precision    recall  f1-score   support

           0       1.00      0.85      0.92     23672
           1       0.05      0.88      0.10       234

    accuracy                           0.85     23906
   macro avg       0.53      0.86      0.51     23906
weighted avg       0.99      0.85      0.91     23906



##### 2.2a Random Forest with SMOTE - Train & Test Accuracy (Oversampled Data)

Almost perfect training & test in oversampled data

In [24]:
RF = LR_training(sm1_x, sm1_y.values.ravel(), RF)

Train acc: 1.0  Test acc:0.9955644735452529
[[18953     0]
 [    0 18922]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18953
           1       1.00      1.00      1.00     18922

    accuracy                           1.00     37875
   macro avg       1.00      1.00      1.00     37875
weighted avg       1.00      1.00      1.00     37875


Test acc: 1.0  Test acc:0.9955644735452529
[[4695   24]
 [  18 4732]]
              precision    recall  f1-score   support

           0       1.00      0.99      1.00      4719
           1       0.99      1.00      1.00      4750

    accuracy                           1.00      9469
   macro avg       1.00      1.00      1.00      9469
weighted avg       1.00      1.00      1.00      9469



##### 2.2b Random Forest with SMOTE - Accuracy (Real non-oversampled data)

Almost perfect training & test in oversampled data


In [25]:
LR_test(train_x, train_y.values.ravel(), RF)

Test acc: 0.998243118882289
[[23648    24]
 [   18   216]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     23672
           1       0.90      0.92      0.91       234

    accuracy                           1.00     23906
   macro avg       0.95      0.96      0.96     23906
weighted avg       1.00      1.00      1.00     23906



#### 3 Random Forest with SMOTETomek, ADASYN

Random Forest + SMOTE has significant result -> model decided to use Random Forest & try other oversampling technique

##### 3.1a Random Forest with SMOTETomek - Train & Test Accuracy (Oversampled Data)

Better than SMOTE

In [26]:
RF = LR_training(sm2_x, sm2_y.values.ravel(), RF)

Train acc: 1.0  Test acc:0.9979885665890323
[[18934     0]
 [    0 18848]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18934
           1       1.00      1.00      1.00     18848

    accuracy                           1.00     37782
   macro avg       1.00      1.00      1.00     37782
weighted avg       1.00      1.00      1.00     37782


Test acc: 1.0  Test acc:0.9979885665890323
[[4669   11]
 [   8 4758]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4680
           1       1.00      1.00      1.00      4766

    accuracy                           1.00      9446
   macro avg       1.00      1.00      1.00      9446
weighted avg       1.00      1.00      1.00      9446



##### 3.1b Random Forest with SMOTETomek - Accuracy (Real non-oversampled data)

Almost perfect training & test in oversampled data
Better Precision, but lower recall and F1

In [27]:
LR_test(train_x, train_y.values.ravel(), RF)

Test acc: 0.9974483393290388
[[23655    17]
 [   44   190]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     23672
           1       0.92      0.81      0.86       234

    accuracy                           1.00     23906
   macro avg       0.96      0.91      0.93     23906
weighted avg       1.00      1.00      1.00     23906



##### 3.2a Random Forest with ADASYN - Train & Test Accuracy (Oversampled Data)

Better than SMOTE

In [28]:
RF = LR_training(sm_x, sm_y.values.ravel(), RF)

Train acc: 1.0  Test acc:0.9952431289640592
[[18957     0]
 [    0 18880]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18957
           1       1.00      1.00      1.00     18880

    accuracy                           1.00     37837
   macro avg       1.00      1.00      1.00     37837
weighted avg       1.00      1.00      1.00     37837


Test acc: 1.0  Test acc:0.9952431289640592
[[4688   27]
 [  18 4727]]
              precision    recall  f1-score   support

           0       1.00      0.99      1.00      4715
           1       0.99      1.00      1.00      4745

    accuracy                           1.00      9460
   macro avg       1.00      1.00      1.00      9460
weighted avg       1.00      1.00      1.00      9460



##### 3.2b Random Forest with ADASYN - Accuracy (Real non-oversampled data)

In [29]:
LR_test(train_x, train_y.values.ravel(), RF)

Test acc: 0.998117627373881
[[23645    27]
 [   18   216]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     23672
           1       0.89      0.92      0.91       234

    accuracy                           1.00     23906
   macro avg       0.94      0.96      0.95     23906
weighted avg       1.00      1.00      1.00     23906



In [30]:
# Manipulate sm_x and sm_y for final selected oversampled data
sm_x, sm_y = sm1_x, sm1_y

In [31]:
# Retrain using selected oversampled data
RF = LR_training(sm_x, sm_y, RF)
LR_test(train_x, train_y.values.ravel(), RF)

C:\Users\Edwin Sanjaya\AppData\Local\Temp\ipykernel_1296\528896171.py:52: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  LR_model.fit(x_train, y_train)


Train acc: 1.0  Test acc:0.9955644735452529
[[18953     0]
 [    0 18922]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18953
           1       1.00      1.00      1.00     18922

    accuracy                           1.00     37875
   macro avg       1.00      1.00      1.00     37875
weighted avg       1.00      1.00      1.00     37875


Test acc: 1.0  Test acc:0.9955644735452529
[[4695   24]
 [  18 4732]]
              precision    recall  f1-score   support

           0       1.00      0.99      1.00      4719
           1       0.99      1.00      1.00      4750

    accuracy                           1.00      9469
   macro avg       1.00      1.00      1.00      9469
weighted avg       1.00      1.00      1.00      9469

Test acc: 0.998243118882289
[[23648    24]
 [   18   216]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     23672
           1       0.90      0.9

#### 4. Recursive Feature Elimintation

##### 4.1a Random Forest with SMOTE + RFE - Train & Test Accuracy (Oversampled Data)

Slight improvement

In [32]:
rfe_columns = RFE_analysis(sm_x, sm_y, RF)
sm_x_rfe = sm_x[sm_x.columns[rfe_columns]]
RF = LR_training(sm_x_rfe, sm_y, RF)

Index(['AGE', 'atm_count_rate', 'atm_tx_rate', 'avg_amt_nontw_nonntd',
       'avg_amt_nontw_ntd', 'avg_amt_tw_nonntd', 'avg_amt_tw_ntd',
       'avg_freq_nontw_nonntd', 'avg_freq_nontw_ntd', 'avg_freq_tw_nonntd',
       ...
       'total_amt_tw_nonntd', 'total_amt_tw_ntd', 'total_asset',
       'total_asset_x', 'total_asset_y', 'total_freq_nontw_nonntd',
       'total_freq_nontw_ntd', 'total_freq_tw_nonntd', 'total_freq_tw_ntd',
       'trade_sum'],
      dtype='object', length=103)
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True False
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True


C:\Users\Edwin Sanjaya\AppData\Local\Temp\ipykernel_1296\528896171.py:52: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  LR_model.fit(x_train, y_train)


Train acc: 1.0  Test acc:0.9957756890907171
[[18953     0]
 [    0 18922]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18953
           1       1.00      1.00      1.00     18922

    accuracy                           1.00     37875
   macro avg       1.00      1.00      1.00     37875
weighted avg       1.00      1.00      1.00     37875


Test acc: 1.0  Test acc:0.9957756890907171
[[4695   24]
 [  16 4734]]
              precision    recall  f1-score   support

           0       1.00      0.99      1.00      4719
           1       0.99      1.00      1.00      4750

    accuracy                           1.00      9469
   macro avg       1.00      1.00      1.00      9469
weighted avg       1.00      1.00      1.00      9469



In [33]:
train_x_rfe = train_x[train_x.columns[rfe_columns]]
attribute_list = list(train_x_rfe.columns)
for i, v in enumerate(attribute_list):
    print(f'{i} {v}')

# Training Evaluation: Without RFE
# LR_test(train_x, train_y, RF)

0 AGE
1 atm_count_rate
2 atm_tx_rate
3 avg_amt_nontw_nonntd
4 avg_amt_nontw_ntd
5 avg_amt_tw_nonntd
6 avg_amt_tw_ntd
7 avg_freq_nontw_nonntd
8 avg_freq_nontw_ntd
9 avg_freq_tw_nonntd
10 avg_freq_tw_ntd
11 cb_count_rate
12 cb_tx_rate
13 count_assets
14 cr_1_avg_tx
15 cr_1_count_tx
16 cr_1_std_tx
17 cr_1_total_tx
18 cr_1_tx_per_day
19 cr_2_avg_tx
20 cr_2_count_tx
21 cr_2_std_tx
22 cr_2_total_tx
23 cr_2_tx_per_day
24 cr_3_avg_tx
25 cr_3_count_tx
26 cr_3_std_tx
27 cr_3_total_tx
28 cr_3_tx_per_day
29 cucah_quarter1_mean
30 cucah_quarter1_std
31 cucah_quarter1_sum
32 cucah_quarter2_mean
33 cucah_quarter2_std
34 cucah_quarter2_sum
35 cucah_quarter3_mean
36 cucah_quarter3_std
37 cucah_quarter3_sum
38 cucah_quarter4_mean
39 cucah_quarter4_std
40 cucah_quarter4_sum
41 cucah_total_mean
42 cucah_total_std
43 cucah_total_sum
44 db_1_avg_tx
45 db_1_count_tx
46 db_1_std_tx
47 db_1_total_tx
48 db_1_tx_per_day
49 db_2_avg_tx
50 db_2_count_tx
51 db_2_std_tx
52 db_2_total_tx
53 db_2_tx_per_day
54 db_3_av

##### 4.2b Random Forest with SMOTE + RFE - Accuracy (Real non-oversampled data)

Slight improvement

In [34]:
LR_test(train_x_rfe, train_y, RF)

Test acc: 0.9983267798878942
[[23648    24]
 [   16   218]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     23672
           1       0.90      0.93      0.92       234

    accuracy                           1.00     23906
   macro avg       0.95      0.97      0.96     23906
weighted avg       1.00      1.00      1.00     23906



In [ ]:
# Fine-tuning test

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

n_estimators = [int(x) for x in np.linspace(start = 300, stop = 500, num = 5)]
max_features = ['sqrt']
max_depth = [int(x) for x in np.linspace(20, 50, num = 4)]
max_depth.append(None)
min_samples_split = [2, 3, 4]
min_samples_leaf = [1, 2, 3]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

param_grid = {
    'bootstrap': [True],
    'max_depth': max_depth,
    'max_features': max_features,
    'min_samples_leaf': min_samples_leaf,
    'min_samples_split': min_samples_split,
    'n_estimators': n_estimators,
}

RF_search = RandomForestClassifier(random_state=42)
# RF_random = RandomizedSearchCV(estimator = RF_search, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
RF_random = GridSearchCV(estimator = RF_search, param_grid = param_grid,
                           cv = 5, n_jobs = -1, verbose = 2)
RF_random.fit(sm_x_rfe, sm_y)
RF_random.best_params_

In [ ]:
# {'bootstrap': True,
#  'max_depth': 70,
#  'max_features': 'auto',
#  'min_samples_leaf': 1,
#  'min_samples_split': 3,
#  'n_estimators': 300}

# {'bootstrap': True,
#  'max_depth': 50,
#  'max_features': 'sqrt',
#  'min_samples_leaf': 1,
#  'min_samples_split': 3,
#  'n_estimators': 400}

# Use Best Tuning Param
# RF_random = RandomForestClassifier(n_estimators=200, min_samples_split=2, min_samples_leaf=1, max_features='sqrt', max_depth=90, bootstrap=True)
# RF_random.fit(sm_x, sm_y)

# LR_test(ori_train_x_normal, ori_train_y, RF_random)

In [ ]:
LR_test(train_x_rfe, train_y, RF_random)

In [ ]:
test_xdp = pd.read_pickle('xdp_x_test.pkl')

In [ ]:
# Test
test = df_from_csv('submit_format.csv')
test = test.drop(['probability'], axis=1)
test = test.merge(test_xdp, on='alert_key', how='left')
test = test.merge(ccba, on='cust_id', how='left')
test = test.merge(cdtx, on='alert_key', how='left')
test = test.merge(remit, on='alert_key', how='left')
test = test.drop(['alert_key', 'cust_id', 'date'], axis=1)
test['bad_value'] = test.isnull().sum(axis=1) + (test == 0).astype(int).sum(axis=1)
test.iloc[:, :] = test.iloc[:, :].fillna(0)
test = test.drop(['bad_value'], axis=1)
test = test.reindex(sorted(test.columns), axis=1)
test = normalize_mms(test)
test

In [ ]:
attribute_list = list(train_x_rfe.columns)
for i, v in enumerate(attribute_list):
    print(f'{i} {v} {test.columns[i]}')

In [ ]:
# #SMOTE
test = test.drop(['sd_amt_tw_nonntd','sd_freq_tw_nonntd', 'total_freq_tw_nonntd'], axis=1)
test['occupation_code_6'] = 0
test['occupation_code_8'] = 0

# #ADASYN
# test = test.drop(['sd_amt_tw_nonntd','sd_freq_tw_nonntd'], axis=1)
# test['occupation_code_8'] = 0

In [ ]:
# Handle Missing Column

test = test.reindex(sorted(test.columns), axis=1)
attribute_list = list(train_x_rfe.columns)
for i, v in enumerate(attribute_list):
    print(f'{i} {v} {test.columns[i]}')

In [ ]:
test

In [ ]:
y_test = RF.predict_proba(test)
# y_test = RF_random.predict_proba(test)
y_test = pd.DataFrame(y_test, columns=['probability0','probability'])
y_test = y_test.drop(['probability0'], axis=1)
# y_test['probability'] = round(y_test['probability'], 6)
y_test.index = test.index
y_test

In [ ]:
y_test.sort_values(by='probability', ascending=True)

In [ ]:
y_test.loc[y_test['probability'] == 0, 'probability'] = 0
y_test.sort_values(by='probability', ascending=True)

In [ ]:
xy_test = pd.concat([test, y_test], axis=1)
submit = df_from_csv('submit_format.csv')
submit = submit.drop(columns=['probability'])
submit = pd.concat([submit, xy_test], axis=1)
submit = submit[['alert_key','probability']]
submit['probability'] = submit['probability'].fillna(0)
submit = submit.sort_values(by='probability', ascending=False)
submit.to_csv('final_submit.csv')
submit

In [ ]:
submit.isnull().any()

In [ ]:
submit.loc[submit.duplicated(keep=False)]